In [1]:
import random

from distributed_processing.serializers import DummySerializer
from distributed_processing.client import Client
from distributed_processing.filesystem_connector import FileSystemConnector


In [2]:
CURRO = True

if CURRO:
    NS_PATH ="G:\\fs_namespaces\\prueba_distribuida"
    #NS_PATH ="C:\\fs_namespaces\\prueba_distribuida"

else:
    NS_PATH = "/home/augusto/python/notebooks/fs_namespaces/prueba_distribuida"


In [3]:
import logging
logging.getLogger("distributed_processing").setLevel(logging.DEBUG)
#logging.getLogger("distributed_processing.filesystem_connector").setLevel(logging.DEBUG)

In [4]:
fs_connector = FileSystemConnector(NS_PATH)
fs_connector.with_watchdog=True

client = Client(DummySerializer(), fs_connector, check_registry="cache")

registry_lock locked: tried 1 times
registry_lock unlocked
Client with id: fs_client_8
Results queue: fs_client_8_responses


In [5]:
fs_connector.methods_registry()

registry_lock locked: tried 1 times
registry_lock unlocked


{'add': ['requests_cola_1'],
 'div': ['requests_cola_1'],
 'eval_py_function': ['requests_py_eval'],
 'mul': ['requests_cola_1']}

In [6]:
y = client.rpc_async("add", [1, 0])

Sent request with id: fs_client_8:1 to queue requests_cola_1


In [7]:
y.get()

1

In [8]:
client.check_registry ="always"

try:
    y = client.rpc_async("fake", [1, 0])
except Exception as e:
    print(e)

'method_queues_fake'


In [9]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

try:
    y.get()
except Exception as e:
    print(e)


Sent request with id: fs_client_8:2 to queue requests_cola_1


Error -32601 : The method does not exist/is not available.

 


In [10]:
client.check_registry ="never"
client.set_default_queue("cola_1")

y = client.rpc_async("fake", [1, 0])

y.safe_get(default="Esto es un error")

Sent request with id: fs_client_8:3 to queue requests_cola_1


'Esto es un error'

In [11]:
client.check_registry ="cache"

In [12]:
x = client.rpc_async("div", [1, 0])

Sent request with id: fs_client_8:4 to queue requests_cola_1


In [13]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32603 : Internal RPC error.

 Traceback (most recent call last):
  File "G:\python\distributed_processing\python\distributed_processing\worker.py", line 149, in _process_single_request
    result = func[request["method"]](*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\agarcia\AppData\Local\Temp\ipykernel_11212\1866336200.py", line 8, in div
    return x / y
           ~~^~~
ZeroDivisionError: division by zero



In [14]:
client.rpc_sync("add", [1, 1])

Sent request with id: fs_client_8:5 to queue requests_cola_1


2

In [15]:
def f(x,y): return x + y

y = client.rpc_async_fn(f, [1, 2.0])

Sent request with id: fs_client_8:6 to queue requests_py_eval


In [16]:
y.get()

3.0

In [17]:
client.rpc_sync_fn(f, [3.0, 2.0])

Sent request with id: fs_client_8:7 to queue requests_py_eval


5.0

In [18]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)
    fs.append(client.rpc_async(t[0], t[1]))

Sent request with id: fs_client_8:8 to queue requests_cola_1


('mul', [0.9310778652941694, 0.42680490812944827], {})
('add', [0.19835790227472705, 0.7373592994891559], {})


Sent request with id: fs_client_8:9 to queue requests_cola_1
Sent request with id: fs_client_8:10 to queue requests_cola_1
Sent request with id: fs_client_8:11 to queue requests_cola_1


('div', [0.05930833616290898, 0.7464618337145831], {})
('add', [0.8589317330859112, 0.8951106942163912], {})


Sent request with id: fs_client_8:12 to queue requests_cola_1


('div', [0.8867805178873676, 0.2519812252955823], {})
('div', [0.11145505278683998, 0.9775709089785757], {})


Sent request with id: fs_client_8:13 to queue requests_cola_1
Sent request with id: fs_client_8:14 to queue requests_cola_1


('mul', [0.5930020731292368, 0.936446310536412], {})
('div', [0.6917438450654931, 0.6441394127811294], {})


Sent request with id: fs_client_8:15 to queue requests_cola_1
Sent request with id: fs_client_8:16 to queue requests_cola_1
Sent request with id: fs_client_8:17 to queue requests_cola_1


('div', [0.28048148334449585, 0.3619924692726658], {})
('add', [0.7928420916667471, 0.2877636780706918], {})


In [19]:
[f.get() for f in fs]

[0.3973886027582408,
 0.935717201763883,
 0.0794526035815866,
 1.7540424273023025,
 3.5192325017355746,
 0.11401224378014159,
 0.5553146035223174,
 1.0739039272241195,
 0.7748268462823381,
 1.080605769737439]

In [20]:
fs = client.rpc_multi_async(tp)

Sent request with id: fs_client_8:18 to queue requests_cola_1
Sent request with id: fs_client_8:19 to queue requests_cola_1
Sent request with id: fs_client_8:20 to queue requests_cola_1
Sent request with id: fs_client_8:21 to queue requests_cola_1
Sent request with id: fs_client_8:22 to queue requests_cola_1
Sent request with id: fs_client_8:23 to queue requests_cola_1
Sent request with id: fs_client_8:24 to queue requests_cola_1
Sent request with id: fs_client_8:25 to queue requests_cola_1
Sent request with id: fs_client_8:26 to queue requests_cola_1
Sent request with id: fs_client_8:27 to queue requests_cola_1


In [21]:
[f.status for f in fs]

['PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING',
 'PENDING']

In [22]:
client._fetch_all_available_responses()

In [23]:
client.pending

{'fs_client_8:18': 1743627799.7102692,
 'fs_client_8:19': 1743627799.822434,
 'fs_client_8:20': 1743627799.9109473,
 'fs_client_8:21': 1743627800.0077562,
 'fs_client_8:22': 1743627800.0953689,
 'fs_client_8:23': 1743627800.1807706,
 'fs_client_8:24': 1743627800.2998328,
 'fs_client_8:25': 1743627800.3656628,
 'fs_client_8:26': 1743627800.456509,
 'fs_client_8:27': 1743627800.55219}

In [24]:
[f.get() for f in fs]

[0.3973886027582408,
 0.935717201763883,
 0.0794526035815866,
 1.7540424273023025,
 3.5192325017355746,
 0.11401224378014159,
 0.5553146035223174,
 1.0739039272241195,
 0.7748268462823381,
 1.080605769737439]

In [25]:
fs = client.rpc_batch_async(tp)

Sent batch request with 10 requests to requests_cola_1


In [26]:
[f.get() for f in fs]

[0.3973886027582408,
 0.935717201763883,
 0.0794526035815866,
 1.7540424273023025,
 3.5192325017355746,
 0.11401224378014159,
 0.5553146035223174,
 1.0739039272241195,
 0.7748268462823381,
 1.080605769737439]

In [27]:
client.rpc_batch_sync(tp)

Sent batch request with 10 requests to requests_cola_1


[0.3973886027582408,
 0.935717201763883,
 0.0794526035815866,
 1.7540424273023025,
 3.5192325017355746,
 0.11401224378014159,
 0.5553146035223174,
 1.0739039272241195,
 0.7748268462823381,
 1.080605769737439]

In [28]:
fs =[]
tp = []
N = 10
for i in range(N):
    fn = random.choice(("add", "mul", "div", "fake"))
    t = (fn, [random.random(),random.random()], {})
    print(t)
    tp.append(t)

('add', [0.5985671053585057, 0.5513488599304533], {})
('mul', [0.8356012673913551, 0.6316160175591796], {})
('add', [0.24356020420305124, 0.04892533937642707], {})
('div', [0.7466443719306405, 0.19060459031071386], {})
('mul', [0.17706692739590946, 0.18054034287323262], {})
('fake', [0.6929607413511351, 0.6884459519362975], {})
('mul', [0.7018687976109835, 0.2554804639339521], {})
('mul', [0.8786140168102163, 0.3170585959756841], {})
('add', [0.42219125943081137, 0.1190657562767865], {})
('fake', [0.9013220479934159, 0.23136931371674507], {})


In [29]:
tp

[('add', [0.5985671053585057, 0.5513488599304533], {}),
 ('mul', [0.8356012673913551, 0.6316160175591796], {}),
 ('add', [0.24356020420305124, 0.04892533937642707], {}),
 ('div', [0.7466443719306405, 0.19060459031071386], {}),
 ('mul', [0.17706692739590946, 0.18054034287323262], {}),
 ('fake', [0.6929607413511351, 0.6884459519362975], {}),
 ('mul', [0.7018687976109835, 0.2554804639339521], {}),
 ('mul', [0.8786140168102163, 0.3170585959756841], {}),
 ('add', [0.42219125943081137, 0.1190657562767865], {}),
 ('fake', [0.9013220479934159, 0.23136931371674507], {})]

In [30]:
client.check_registry ="never"
client.set_default_queue("cola_1")

fs = client.rpc_batch_async(tp)

Sent batch request with 10 requests to requests_cola_1


In [31]:
[f.safe_get() for f in fs]

[1.149915965288959,
 0.5277791447771308,
 0.2924855435794783,
 3.917242342975576,
 0.03196772378356728,
 None,
 0.17931376603441918,
 0.2785721265744033,
 0.5412570157075979,
 None]

In [32]:
client.responses

{}

In [33]:
client.check_registry ="cache"

In [34]:
try:
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [35]:
try:    
    x = client.rpc_batch_sync(tp, timeout=5)
except Exception as e:
    print(e)

Method fake does not exist/is not available.


In [36]:
client.check_registry="never"
client.set_default_queue("cola_1")

x = client.rpc_async("kk", [1, 0])

Sent request with id: fs_client_8:58 to queue requests_cola_1


In [37]:
try:
    x.get()
except Exception as e:
    print(e)

Error -32601 : The method does not exist/is not available.

 


In [38]:
y = client.rpc_async("add", [1, 0])

Sent request with id: fs_client_8:59 to queue requests_cola_1


In [39]:
y.get(5)

1

In [40]:
def f(x,y): return x + y

client.check_registry="never"
y = client.rpc_async_fn(f, [1, 2.0])
try:
    y.get()
except Exception as e:
    print(e)

Sent request with id: fs_client_8:60 to queue requests_cola_1


Error -32601 : The method does not exist/is not available.

 
